In [2]:
import jsonlines
import time
import datetime
import json


def retrive_data(filename):
    twitter = []
    with open(filename,encoding='utf-8') as f:
        for item in jsonlines.Reader(f):
            twitter.append(item)
    totalData = []
    for tweets in twitter:  #under one event
        TweetList = []
        for tweet in tweets:
            tweetText = tweet['text']
            tweetId = tweet['id']
            createTime = time.mktime(datetime.datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S +0000 %Y").timetuple())
            TweetList.append({'id':tweetId, 'text':tweetText, 'time': createTime})
        TweetList = sorted(TweetList, key=lambda k: k['time'])        #from early to late
        TotalTimeLine = TweetList[-1]['time']-TweetList[0]['time']
        
        #we only consider source and reply text currently
        source_reply =  ' [SEP]'.join([t['text'] for t in TweetList])
        totalData.append(source_reply)
    return totalData
def retrive_label(filename):
    id_label = {}
    rumor = []
    label = []
    with open(filename,'r') as f:
        temp = json.loads(f.read())
        id_label = temp
    for i in id_label.values():
        rumor.append(i)
    for key,val in id_label.items():
        if val == 'rumour':
            id_label[key] = 0
        if val == 'non-rumour':
            id_label[key] = 1
    for i in id_label.values():
        label.append(i)
    return id_label,label

In [3]:
x_dev = retrive_data('dev.data.jsonl')
x_train = retrive_data('train.data.jsonl')
x_test = retrive_data('test.data.jsonl')
y_dev,dev_label = retrive_label('dev.label.json')
y_train,train_label = retrive_label('train.label.json')

In [4]:
import pandas as pd
train_pd = {}
dev_pd = {}
test_pd = {}
train_pd['text'] = [x for x in x_train]
train_pd['label'] = [i for i in train_label]
dev_pd['text'] = [x for x in x_dev]
dev_pd['label'] = [i for i in dev_label]
test_pd['text'] = [x for x in x_test]

train_df = pd.DataFrame(train_pd)
dev_df = pd.DataFrame(dev_pd)
test_df = pd.DataFrame(test_pd)

In [5]:
import re
import itertools
import emoji
def clean_dataset(text):
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    #text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    #text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 4 or fewer letters
    #text = re.sub(r'\b\w{1,4}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    #text= ''.join(c for c in text if c <= '\uFFFF') 
    #text = text.strip()
    # Remove misspelling words
    #text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    #text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [6]:
train_df['text'] = train_df['text'].apply(clean_dataset)
dev_df['text'] =dev_df['text'].apply(clean_dataset)
test_df['text'] = test_df['text'].apply(clean_dataset)

In [7]:
training_label = train_df['label']

In [8]:
total_sample = 2 * sum(training_label)
sweight = [2 if i==0 else 1 for i in training_label]

In [9]:
total_sample

6116

In [30]:
len(source_tokens)

127

In [11]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer

class SSTDataset(Dataset):

    def __init__(self, filename, sour_maxlen, reply_maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = filename

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.sour_maxlen = sour_maxlen
        self.reply_maxlen = reply_maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df['text'][index]
        label = self.df['label'][index]

        #Preprocessing the text to be suitable for BERT
        tweets = sentence.split('[SEP]')
        source_tokens = self.tokenizer.tokenize(tweets[0])
        source_tokens = ['[CLS]'] + source_tokens
        if len(source_tokens) < self.sour_maxlen:
            source_tokens = source_tokens + ['[PAD]' for _ in range((self.sour_maxlen-1) - len(source_tokens))] #Padding sentences
        source_tokens = source_tokens + ['[SEP]']
        
        reply_tokens = []
        for i in range(1,len(tweets)):
            reply = self.tokenizer.tokenize(tweets[i])
            reply = reply + ['[SEP]']
            if (len(reply) + len(reply_tokens)) < 328:
                reply_tokens = reply_tokens + reply
        if len(source_tokens) < 328:
            reply_tokens = reply_tokens + ['[PAD]' for _ in range(328 - len(reply_tokens))] #Padding sentences

        tokens = source_tokens + reply_tokense
        
        

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [12]:
from torch.utils.data import DataLoader

#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = SSTDataset(filename = train_df, sour_maxlen = 60, reply_maxlen = 328)
dev_set = SSTDataset(filename = dev_df, sour_maxlen = 60, reply_maxlen = 328)

from torch.utils.data.sampler import  WeightedRandomSampler
wsampler = WeightedRandomSampler(sweight, num_samples=total_sample, replacement=True)
#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 64, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 64, num_workers = 2)

print("Done preprocessing training and development data.")

Done preprocessing training and development data.


In [13]:
import torch
import torch.nn as nn
from transformers import BertModel

class rumour_classifier(nn.Module):

    def __init__(self):
        super(rumour_classifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [14]:

print("Creating the rumor classifier, initialised with pretrained BERT-BASE parameters...")
net = rumour_classifier()
net.cuda() #Enable gpu support for the model
print("Done creating the rumor classifier.")

Creating the rumor classifier, initialised with pretrained BERT-BASE parameters...
Done creating the rumor classifier.


In [15]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [19]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

In [20]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [21]:
cuda_flag = torch.cuda.is_available()
cuda_flag

True

In [ ]:
num_epoch = 20#epoch 是什么


#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch)